In [8]:
'''
Here's a CS50-level recursion problem that's both engaging and instructive:

Write a program that implements a "word ladder" finder with these requirements:

1. Given a starting word, find all "child" words that can be made by:
   - Adding one letter anywhere in the word
   - Removing one letter from anywhere in the word
   - Changing one letter to another letter
2. The program should recursively explore these transformations up to a specified depth
3. Only return words that appear in a provided dictionary/word list
4. Print each valid word found, indented to show its "depth" in the transformation tree

Example output for starting word "cat" with depth 2 might look like:
```
cat
  bat
    bat's
    beat
    boat
  cat's
  chat
  coat
  cut
    curt
    cute
```

This problem practices:
- Recursion with meaningful branching
- String manipulation
- Working with sets/dictionaries
- Tree-like data structures
- Basic file I/O (for the dictionary)

To help get started, you can use this small word list for testing:
```python
WORD_LIST = {'cat', 'bat', "bat's", 'beat', 'boat', "cat's", 'chat', 'coat', 'cut', 'curt', 'cute'}
```

The recursion here is non-trivial because:
- Each word can transform in multiple ways
- You need to track visited words to avoid cycles
- The branching factor increases with word length
- You need to manage the depth of recursion

Would you like to try solving this? I can provide hints or clarification if needed.

Here's a pseudocode outline for the word ladder problem:

```
Function find_word_ladder(current_word, depth, max_depth, dictionary, visited_words):
    # Base cases
    If depth > max_depth:
        Return
    If current_word in visited_words:
        Return
    
    # Print current word with proper indentation
    Print("  " * depth + current_word)
    
    # Add current word to visited set
    Add current_word to visited_words
    
    # For each possible transformation of current_word:
    # 1. Adding a letter
    For each position in range(0 to len(current_word) + 1):
        For each letter in alphabet:
            new_word = current_word with letter inserted at position
            If new_word in dictionary:
                find_word_ladder(new_word, depth + 1, max_depth, dictionary, visited_words)
    
    # 2. Removing a letter
    For each position in range(0 to len(current_word)):
        new_word = current_word with letter at position removed
        If new_word in dictionary:
            find_word_ladder(new_word, depth + 1, max_depth, dictionary, visited_words)
    
    # 3. Changing a letter
    For each position in range(0 to len(current_word)):
        For each letter in alphabet:
            new_word = current_word with letter at position changed
            If new_word in dictionary:
                find_word_ladder(new_word, depth + 1, max_depth, dictionary, visited_words)

# Main program
starting_word = get_input_from_user()
max_depth = get_depth_from_user()
visited_words = empty_set()
find_word_ladder(starting_word, 0, max_depth, WORD_LIST, visited_words)
```

Key points about this structure:
1. The recursion happens when a valid transformation is found
2. We track visited words to avoid cycles (e.g., cat -> bat -> cat)
3. The depth parameter helps with:
   - Controlling recursion depth
   - Proper indentation of output
4. Each recursive call represents one step in the transformation

'''
import string
ALPHABET=list(string.ascii_lowercase)+["'s"]
WORD_LIST = {'cat', 'bat', "bat's", 'beat', 'boat', "cat's", 'chat', 'coat', 'cut', 'curt', 'cute'}


def find_word_ladder(current_word, depth, max_depth, dictionary, visited_words):
    # Base cases
    if depth > max_depth:
        return
    if current_word in visited_words:
        return
    
    # Print current word with proper indentation
    print("  " * depth + current_word)
    
    # Add current word to visited set
    visited_words.add(current_word)
    
    # For each possible transformation of current_word:
    # 1. Adding a letter
    for position in range(len(current_word) + 2):
        for letter in ALPHABET:
            new_word = current_word[:position]+letter+current_word[position:]
            if new_word in dictionary:
                find_word_ladder(new_word, depth + 1, max_depth, dictionary, visited_words)
    
    # 2. Removing a letter
    for position in range(len(current_word)+1):
        new_word = current_word[:position]+current_word[position+1:]
        if new_word in dictionary:
            find_word_ladder(new_word, depth + 1, max_depth, dictionary, visited_words)
    
    # 3. Changing a letter
    for position in range(len(current_word)+1):
        for letter in ALPHABET:
            new_word = current_word[:position]+letter+current_word[position+1:]
            if new_word in dictionary:
                find_word_ladder(new_word, depth + 1, max_depth, dictionary, visited_words)


# Main program
print('Starting Word: ')
starting_word = input()
print('Max Depth: ')
max_depth = int(input())
visited_words = set()
find_word_ladder(starting_word, 0, max_depth, WORD_LIST, visited_words)

Starting Word: 
cat
Max Depth: 
2
cat
  chat
    coat
  cat's
    bat's
  bat
    beat
    boat
  cut
    curt
    cute


In [10]:
'''
You've identified a key issue - the program isn't finding the shortest paths to words. This happens because the current implementation explores paths in a depth-first manner and marks words as visited as soon as they're found, even if we later discover a shorter path to them.
Here's an improved version that uses a breadth-first approach to ensure we find the shortest paths:

Key improvements:

Uses a queue (breadth-first search) instead of recursion
Tracks the depth at which we first see each word
Only processes a word again if we find a shorter path to it
Maintains a parent map that could be used to reconstruct paths (though not used in current output)

This version will now correctly identify that "coat" is reachable from "cat" in one step rather than showing it at depth 2.
Additional possible improvements:

Add path reconstruction to show how we reached each word
Group words by depth in the output
Add optimization to generate only valid dictionary words
Add memoization for frequently computed transformations


'''
import string
from collections import defaultdict, deque
ALPHABET = list(string.ascii_lowercase) + ["'s"]
WORD_LIST = {'cat', 'bat', "bat's", 'beat', 'boat', "cat's", 'chat', 'coat', 'cut', 'curt', 'cute'}

def find_word_ladder(starting_word, max_depth, dictionary):
    # Queue stores (word, depth) pairs
    queue = deque([(starting_word, 0)])
    # Keep track of how we reached each word for path reconstruction
    parent_map = {starting_word: None}
    # Track minimum depth at which we've seen each word
    depth_map = {starting_word: 0}
    
    # Process all words up to max_depth
    while queue:
        current_word, depth = queue.popleft()
        
        # Print current word with proper indentation
        print("  " * depth + current_word)
        
        if depth >= max_depth:
            continue
            
        # Generate all possible transformations
        transformations = set()
        
        # 1. Adding a letter
        for position in range(len(current_word) + 1):
            for letter in ALPHABET:
                new_word = current_word[:position] + letter + current_word[position:]
                transformations.add(new_word)
        
        # 2. Removing a letter
        for position in range(len(current_word)):
            new_word = current_word[:position] + current_word[position+1:]
            transformations.add(new_word)
        
        # 3. Changing a letter
        for position in range(len(current_word)):
            for letter in ALPHABET:
                new_word = current_word[:position] + letter + current_word[position+1:]
                transformations.add(new_word)
        
        # Process valid transformations
        for new_word in transformations:
            if new_word in dictionary:
                new_depth = depth + 1
                # Only process if we haven't seen this word before or if we've found a shorter path
                if new_word not in depth_map or new_depth < depth_map[new_word]:
                    depth_map[new_word] = new_depth
                    parent_map[new_word] = current_word
                    queue.append((new_word, new_depth))

if __name__ == "__main__":
    print('Starting Word: ')
    starting_word = input()
    print('Max Depth: ')
    max_depth = int(input())
    find_word_ladder(starting_word, max_depth, WORD_LIST)


Starting Word: 
cat
Max Depth: 
2
cat
  cut
  cat's
  bat
  coat
  chat
    cute
    curt
    bat's
    beat
    boat


In [11]:
'''Here's a modified version that shows all possible paths to each word:
pythonCopy

For the starting word "cat" with depth 2, the output would look something like:
Copycat
  cut (paths: cat -> cut)
  cat's (paths: cat -> cat's)
  bat (paths: cat -> bat)
  coat (paths: cat -> coat)
  chat (paths: cat -> chat)
    cute (paths: cat -> cut -> cute)
    curt (paths: cat -> cut -> curt)
    bat's (paths: cat -> bat -> bat's OR cat -> cat's -> bat's)
    beat (paths: cat -> bat -> beat)
    boat (paths: cat -> bat -> boat)
Key differences:

Uses a defaultdict to store all valid paths to each word
Shows multiple paths when they exist at the same depth
Still maintains proper depth-first printing order
Shows all equally short paths to reach each word

The interesting part is seeing words like "bat's" that can be reached through multiple valid paths of the same length. This gives us more insight into the connectivity of the word network.
Would you like me to adjust the format of how multiple paths are displayed, or would you like to explore other aspects of the multiple-path approach?'''

import string
from collections import defaultdict, deque

ALPHABET = list(string.ascii_lowercase) + ["'s"]
WORD_LIST = {'cat', 'bat', "bat's", 'beat', 'boat', "cat's", 'chat', 'coat', 'cut', 'curt', 'cute'}

def find_word_ladder(starting_word, max_depth, dictionary):
    # Queue stores (word, path) pairs
    queue = deque([(starting_word, [starting_word])])
    # Track all paths to each word
    paths = defaultdict(set)
    paths[starting_word].add(tuple([starting_word]))
    
    while queue:
        current_word, current_path = queue.popleft()
        depth = len(current_path) - 1
        
        # Print current word with all its paths
        paths_str = [" -> ".join(path) for path in paths[current_word]]
        print("  " * depth + current_word + 
              (" (paths: " + " OR " + " OR ".join(paths_str) + ")" if depth > 0 else ""))
        
        if depth >= max_depth:
            continue
            
        # Generate all possible transformations
        transformations = set()
        
        # 1. Adding a letter
        for position in range(len(current_word) + 1):
            for letter in ALPHABET:
                new_word = current_word[:position] + letter + current_word[position:]
                transformations.add(new_word)
        
        # 2. Removing a letter
        for position in range(len(current_word)):
            new_word = current_word[:position] + current_word[position+1:]
            transformations.add(new_word)
        
        # 3. Changing a letter
        for position in range(len(current_word)):
            for letter in ALPHABET:
                new_word = current_word[:position] + letter + current_word[position+1:]
                transformations.add(new_word)
        
        # Process valid transformations
        for new_word in transformations:
            if new_word in dictionary:
                new_path = tuple(current_path + [new_word])
                if len(new_path) <= max_depth + 1:  # +1 because path includes start
                    paths[new_word].add(new_path)
                    # Only add to queue if this is a new path of the same or shorter length
                    min_existing_len = min([len(p) for p in paths[new_word]])
                    if len(new_path) <= min_existing_len:
                        queue.append((new_word, list(new_path)))

if __name__ == "__main__":
    print('Starting Word: ')
    starting_word = input()
    print('Max Depth: ')
    max_depth = int(input())
    find_word_ladder(starting_word, max_depth, WORD_LIST)
    

Starting Word: 
cat
Max Depth: 
2
cat
  cut (paths:  OR cat -> cut)
  cat's (paths:  OR cat -> cat's)
  bat (paths:  OR cat -> bat)
  coat (paths:  OR cat -> coat)
  chat (paths:  OR cat -> coat -> chat OR cat -> chat)
    cute (paths:  OR cat -> cut -> cute)
    curt (paths:  OR cat -> cut -> curt)
    bat's (paths:  OR cat -> cat's -> bat's OR cat -> bat -> bat's)
    beat (paths:  OR cat -> bat -> beat)
    bat's (paths:  OR cat -> cat's -> bat's OR cat -> bat -> bat's)
    boat (paths:  OR cat -> coat -> boat OR cat -> bat -> boat)
    boat (paths:  OR cat -> coat -> boat OR cat -> bat -> boat)


In [13]:
import string
from collections import defaultdict, deque

ALPHABET = list(string.ascii_lowercase) + ["'s"]
WORD_LIST = {'cat', 'bat', "bat's", 'beat', 'boat', "cat's", 'chat', 'coat', 'cut', 'curt', 'cute'}

def find_word_ladder(starting_word, max_depth, dictionary):
    # Queue stores (word, path) pairs
    queue = deque([(starting_word, [starting_word])])
    # Track all paths to each word and their lengths
    paths = defaultdict(lambda: (float('inf'), set()))  # (min_length, paths_set)
    paths[starting_word] = (1, {tuple([starting_word])})
    
    while queue:
        current_word, current_path = queue.popleft()
        current_depth = len(current_path)
        min_depth, _ = paths[current_word]
        
        # Only process if this is a shortest path
        if current_depth > min_depth:
            continue
            
        depth = len(current_path) - 1
        if depth > 0:  # Don't print paths for starting word
            current_paths = " OR ".join([" -> ".join(path) for path in paths[current_word][1]])
            print("  " * depth + current_word + f" (paths: {current_paths})")
        else:
            print(current_word)
        
        if depth >= max_depth:
            continue
            
        # Generate all possible transformations
        transformations = set()
        
        # 1. Adding a letter
        for position in range(len(current_word) + 1):
            for letter in ALPHABET:
                new_word = current_word[:position] + letter + current_word[position:]
                transformations.add(new_word)
        
        # 2. Removing a letter
        for position in range(len(current_word)):
            new_word = current_word[:position] + current_word[position+1:]
            transformations.add(new_word)
        
        # 3. Changing a letter
        for position in range(len(current_word)):
            for letter in ALPHABET:
                new_word = current_word[:position] + letter + current_word[position+1:]
                transformations.add(new_word)
        
        # Process valid transformations
        for new_word in transformations:
            if new_word in dictionary:
                new_path = tuple(current_path + [new_word])
                new_depth = len(new_path)
                min_existing_depth, existing_paths = paths[new_word]
                
                if new_depth < min_existing_depth:
                    # Found a shorter path, clear existing paths
                    paths[new_word] = (new_depth, {new_path})
                    queue.append((new_word, list(new_path)))
                elif new_depth == min_existing_depth:
                    # Found another path of same length
                    existing_paths.add(new_path)
                    paths[new_word] = (min_existing_depth, existing_paths)

if __name__ == "__main__":
    print('Starting Word: ')
    starting_word = input()
    print('Max Depth: ')
    max_depth = int(input())
    find_word_ladder(starting_word, max_depth, WORD_LIST)

Starting Word: 
cat
Max Depth: 
3
cat
  cut (paths: cat -> cut)
  cat's (paths: cat -> cat's)
  bat (paths: cat -> bat)
  coat (paths: cat -> coat)
  chat (paths: cat -> chat)
    cute (paths: cat -> cut -> cute)
    curt (paths: cat -> cut -> curt)
    bat's (paths: cat -> cat's -> bat's OR cat -> bat -> bat's)
    beat (paths: cat -> bat -> beat)
    boat (paths: cat -> coat -> boat OR cat -> bat -> boat)
